In [1]:
%%capture
!pip install ht CoolProp fluids xlrd xlwt

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xlwt as xw
lista_data = []
import matplotlib.pyplot as plt
import ht,fluids,os,socket
import CoolProp as cp
from IPython.display import Latex
from prop_fluidos_intercambiadores import fluido_intercambiador


$\require{\siunitx}$

Un generador de vapor consiste en un arreglo en línea de tubos, cada uno de diámetro externo $D = \SI{10}{\milli\meter}$ y longitud $L = \SI{1}{\meter}$. Los espaciados longitudinal y transversal son cada uno $S_L = S_T = \SI{20}{\milli\meter}$, mientras los números de filas longitudinales y transversales son       $N_L = 20$ y $N_T = 5$. Agua saturada (líquida) entra en los tubos a una presión de $\SI{250}{\kilo\pascal}$, y su flujo másico se ajusta para asegurar que salga de los tubos como vapor saturado. Considerar que los tubos están dispuestos con su eje en forma vertical.
La ebullición que ocurre en los tubos mantiene una temperatura de la pared del tubo uniforme de $T_s=\SI{400}{\kelvin}$.
Considerar que al banco de tubos ingresan gases de combustión a una velocidad y una temperatura $u_\infty = \SI{10}{\meter/\second}$ y $T_{m,i} = \SI{1200}{\kelvin}$, respectivamente.
Determine el coeficiente de convección promedio del lado del gas, la temperatura de salida del gas, y la rapidez de producción de vapor en \SI{}{\kilogram/\second}.
Las propiedades del gas se pueden aproximar a las del aire atmosférico a una temperatura de $\SI{900}{\kelvin}$.

ŽUKAUSKAS, A. Heat transfer from tubes in crossflow. En Advances in heat transfer. Elsevier, 1972. p. 93-160.

In [7]:
p_agua = 250e3
Ts = 400
Ti_aire=1200
uinf = 10
L=1
D = 10e-3
t = 1e-3
Di = D - 2*t
NL = 20
NT = 5
g=9.8
N_totales = NL*NT
Area_cond = Di*np.pi*L*N_totales
Area_aire = D*L*NT

Tr_aire = 900

rho_aire = cp.CoolProp.PropsSI('D','T',Tr_aire,'P',101.3e3,'Air')

rho_vap_sat = cp.CoolProp.PropsSI('D','Q',1,'P',p_agua,'Water')
rho_agua_sat = cp.CoolProp.PropsSI('D','Q',0,'P',p_agua,'Water')
T_agua_sat = cp.CoolProp.PropsSI('T','Q',0,'P',p_agua,'Water')
T_vap_sat = cp.CoolProp.PropsSI('T','Q',1,'P',p_agua,'Water')
k_agua = cp.CoolProp.PropsSI('L','Q',0,'P',p_agua,'Water')
h_agua = cp.CoolProp.PropsSI('H','Q',0,'P',p_agua,'Water')
h_vapor = cp.CoolProp.PropsSI('H','Q',1,'P',p_agua,'Water')
mu_agua = cp.CoolProp.PropsSI('V','Q',0,'P',p_agua,'Water')
hfg = h_vapor - h_agua

Cp_aire = cp.CoolProp.PropsSI('C','P',101.3e3,'T',Tr_aire,'Air')


In [8]:
Nu_cond = ht.condensation.Nusselt_laminar(T_agua_sat,Ts,rho_vap_sat,rho_agua_sat,k_agua,mu_agua,hfg,L)
h_cond = Nu_cond*k_agua/L
Q_cond = h_cond * Area_cond * (T_agua_sat-Ts)
m_cond = Q_cond / hfg

In [9]:
delta = L/Nu_cond
delta

7.133206337034413e-05

In [10]:
rho_agua_sat*(rho_agua_sat-rho_vap_sat)/(3*mu_agua**2)*g*delta**3

21.95472684314711

In [11]:
(T_vap_sat-Ts)

0.5614064737957847

In [12]:
delta*Area_cond*rho_agua_sat

0.16798553106282743

In [13]:
m_cond*3600

22.294288628872476

In [14]:
Q_aire = Q_cond

In [15]:
To_aire = Ti_aire- Q_aire / (Cp_aire*rho_aire*uinf*Area_aire)
To_aire

1138.5150037368203

In [18]:
Re_crudo = Veloc_crudo * D_interno / nu_crudo
Re_glycol = Veloc_glycol * D_hidraulico_glycol / nu_glycol
Nu_crudo = ht.conv_internal.turbulent_Colburn(Re_crudo,Pr_crudo)
Nu_glycol = ht.conv_internal.turbulent_Colburn(Re_glycol,Pr_glycol)
hc_crudo = Nu_crudo*k_crudo/D_interno
hc_glycol = Nu_glycol*k_glycol/D_hidraulico_glycol
display(Latex(f'$Re_{{crudo}}={Re_crudo:.0f}$,$Nu_{{crudo}}={Nu_crudo:.1f}$, $hc_{{crudo}}={hc_crudo:.0f}$'))
display(Latex(f'$Re_{{glycol}}={Re_glycol:.0f}$,    $Nu_{{glycol}}={Nu_glycol:.1f}$,$hc_{{glycol}}={hc_glycol:.0f}$'))
#print('Coeficiente conveccion aire = %.1f'%Aire.hc)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

COLBURN, Allan P. A method of correlating forced convection heat-transfer data and a comparison with fluid friction. International Journal of Heat and Mass Transfer, 1964, vol. 7, no 12, p. 1359-1384.

In [21]:
k_acero = ht.k_material('stainless steel')
espesor_tubo = 1e-3

In [22]:
Ulimpio = (1/ (hc_crudo*D_interno) + np.log((D_interno+espesor_tubo)/D_interno)/k_acero + 1/(hc_glycol*(D_interno+espesor_tubo)))**-1
#U_calc = 1 / (1/Ulimpio + Rf *(1+Tubo_dext/Tubo_dint))



In [23]:
1/ (np.log((D_interno+espesor_tubo)/D_interno)/k_acero ), (hc_crudo*D_interno) 

(440.2452970082708, 1.2961839923607767)

In [22]:
Ulimpio,hc_crudo

(4.270114968579573, 1531.9786379170362)

In [24]:
# Caida de presion

# Calculo para flujo en tubos
roughness = fluids.roughness_Farshad('Carbon steel, bare', D=D_interno)
roughness_gl = fluids.roughness_Farshad('Carbon steel, bare', D=D_hidraulico_glycol)

f_c = fluids.friction.friction_factor(Re=Re_crudo, eD=roughness/D_interno)
f_c2 = 1.2*(0.0014+0.125/Re_crudo**.32)
f_c_gl = fluids.friction.friction_factor(Re=Re_glycol, eD=roughness_gl/D_hidraulico_glycol)

Delta_p_crudo = f_c*Veloc_crudo**2*Longitud/D_interno/2 
Delta_p_glycol = f_c_gl*Veloc_glycol**2*Longitud/D_hidraulico_glycol/2 

display(Latex(f'$\Delta p_{{crudo}}={Delta_p_crudo}$Pa'))
display(Latex(f'$\Delta p_{{glycol}}={Delta_p_glycol}$Pa'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [25]:
calcular volumen de parafina necesario.... tres veces lo que deposita 5mm en la pared

SyntaxError: invalid syntax (2619805294.py, line 1)

In [28]:
Diam = 25.4e-3
Diam_para = 25.4e-3-2.5e-3

Vol_para = (Diam**2*np.pi/4-Diam_para**2*np.pi/4)*Longitud

In [29]:
Vol_para*1e3

1.5173892516838692